<a href="https://colab.research.google.com/github/AlexBB999/Classwork/blob/master/14_8_Feature_Engineering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import the SQL ALchemy engine
from sqlalchemy import create_engine

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import boxcox
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats.mstats import winsorize
#from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

In [0]:
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [0]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db =  'useducation'

In [0]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [0]:
ed0 = pd.read_sql_query('select * FROM useducation',con=engine)
#educate= engine.execute('SELECT * FROM useducation')
engine.dispose()

In [0]:
ed=ed0.copy()

In [0]:
education_df=ed0.copy()

In [188]:
ed.head(10)

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN
5,1992_COLORADO,COLORADO,1992,NaN,3185173.0,163253.0,1307986.0,1713934.0,3264826.0,1642466.0,1035970.0,NaN,364760.0,7410.0,47588.0,50648.0,45025.0,34533.0,394904.0,160299.0,562613.0,221.023429,272.398433,213.480890,266.419687
6,1992_CONNECTICUT,CONNECTICUT,1992,NaN,3834302.0,143542.0,1342539.0,2348221.0,3721338.0,2148041.0,1142600.0,NaN,48542.0,5731.0,41319.0,38058.0,33691.0,28366.0,304284.0,126917.0,436932.0,226.798480,273.739345,222.417599,264.713904
7,1992_DELAWARE,DELAWARE,1992,NaN,645233.0,45945.0,420942.0,178346.0,638784.0,372722.0,194915.0,NaN,30595.0,463.0,8025.0,8272.0,8012.0,6129.0,67495.0,28338.0,96296.0,217.899972,262.868585,206.389052,261.577921
8,1992_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1992,NaN,709480.0,64749.0,0.0,644731.0,742893.0,329160.0,316679.0,NaN,47272.0,4818.0,6667.0,5832.0,5000.0,3433.0,47009.0,18173.0,70000.0,192.600553,234.924621,178.557612,265.976765
9,1992_FLORIDA,FLORIDA,1992,NaN,11506299.0,788420.0,5683949.0,5033930.0,11305642.0,5166374.0,3410440.0,NaN,1667826.0,31464.0,161701.0,164416.0,142372.0,100835.0,1276685.0,511557.0,1819706.0,213.689751,259.908848,204.882606,262.118986


In [0]:
ed['LOCAL_REVENUE'].max()


12504507.0

In [0]:
ed.describe()

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.000000,1.229000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.229000e+03,1.280000e+03,1319.000000,1360.000000,1361.000000,1361.000000,1361.00000,1.361000e+03,1.361000e+03,1.319000e+03,536.000000,532.000000,533.000000,498.000000
mean,2004.433646,9.159308e+05,9.092082e+06,7.663723e+05,4.216553e+06,4.109157e+06,9.196681e+06,4.762966e+06,2.680331e+06,4.292046e+05,9.027693e+05,17601.614102,63746.761029,64538.927259,64271.057311,54268.92432,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132
std,7.393983,1.065280e+06,1.174519e+07,1.145242e+06,5.543072e+06,5.482971e+06,1.197813e+07,6.293004e+06,3.353349e+06,5.340693e+05,1.327562e+06,30016.166447,80205.138143,80293.387625,78910.545776,67889.27623,6.436384e+05,3.077056e+05,9.707031e+05,10.221511,10.159722,7.769616,6.803725
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,1.270800e+04,0.000000,544.000000,633.000000,437.000000,311.00000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102
25%,1998.000000,2.583140e+05,2.186305e+06,1.893540e+05,1.153097e+06,7.158345e+05,2.165404e+06,1.168032e+06,6.357900e+05,1.028310e+05,1.815645e+05,2021.000000,11745.000000,13739.000000,13342.000000,10919.00000,1.106260e+05,5.147100e+04,1.810650e+05,229.694352,272.761598,214.663401,259.533171
50%,2004.000000,6.483130e+05,5.079546e+06,4.033765e+05,2.537074e+06,2.055780e+06,5.234506e+06,2.657452e+06,1.525406e+06,2.715960e+05,5.102595e+05,8106.000000,41597.500000,43272.000000,43339.000000,36735.00000,3.457750e+05,1.642600e+05,5.503420e+05,237.238552,280.618803,220.416034,265.010912
75%,2011.000000,1.014528e+06,1.085985e+07,8.289660e+05,5.080939e+06,4.768680e+06,1.074519e+07,5.568028e+06,3.229651e+06,5.186000e+05,9.668515e+05,22295.000000,73865.250000,75481.000000,76566.000000,67460.00000,6.119050e+05,2.905020e+05,9.282755e+05,241.995486,285.347428,223.999337,268.190121
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,1.022366e+07,250911.000000,530531.000000,493415.000000,500143.000000,498403.00000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130


Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

What is the correlation between the overall score variable and the 1st principal component?

If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

**From Sample Solution**


In [0]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = education_df["STATE"].unique()

for state in states:
    education_df.loc[education_df["STATE"] == state, fill_list] = education_df.loc[education_df["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
education_df.dropna(inplace=True)

In [0]:
education_df["overall_score"] = (education_df["GRADES_4_G"]*((education_df["AVG_MATH_4_SCORE"] + education_df["AVG_READING_4_SCORE"])*0.5) + education_df["GRADES_8_G"]
                                 * ((education_df["AVG_MATH_8_SCORE"] + education_df["AVG_READING_8_SCORE"])*0.5))/(education_df["GRADES_4_G"] + education_df["GRADES_8_G"])

In [203]:
education_df[["overall_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overall_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


In [216]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = education_df[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
education_df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


In [217]:
education_df[["overall_score", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()


,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


**End example solutuon stuff**

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


0 - **Clean up the data-- Missing Values**

1.   Average grades handled seperately since so many mssing values?



In [0]:
fill_list=list(ed)

In [0]:
fill_list2=['ENROLL',
 'TOTAL_REVENUE',
 'FEDERAL_REVENUE',
 'STATE_REVENUE',
 'LOCAL_REVENUE',
 'TOTAL_EXPENDITURE',
 'INSTRUCTION_EXPENDITURE',
 'SUPPORT_SERVICES_EXPENDITURE',
 'OTHER_EXPENDITURE',
 'CAPITAL_OUTLAY_EXPENDITURE',
 'GRADES_PK_G',
 'GRADES_KG_G',
 'GRADES_4_G',
 'GRADES_8_G',
 'GRADES_12_G',
 'GRADES_1_8_G',
 'GRADES_9_12_G',
 'GRADES_ALL_G','AVG_MATH_4_SCORE',
 'AVG_MATH_8_SCORE',
 'AVG_READING_4_SCORE',
 'AVG_READING_8_SCORE']

In [0]:
fill_sub=['AVG_MATH_4_SCORE',
 'AVG_MATH_8_SCORE',
 'AVG_READING_4_SCORE',
 'AVG_READING_8_SCORE']

for col in fill_list2:
  ed[col]=ed[col].interpolate(limit_direction ='both')

In [0]:
states = ed["STATE"].unique()
for state in states:
    ed.loc[ed["STATE"] == state, fill_list2] = ed.loc[ed["STATE"] == state, fill_list2].interpolate()

In [0]:
ed.dropna(inplace=True)

In [0]:
#ed.info()

In [0]:
#ed.isnull().sum()

**Histograms for all 22 Numeric Features** (skipped year as categorical)

only a few look normal

there are outliers



plt.figure(figsize=(10,20))
#plt.subplots_adjust(right=1,top=1,bottom=.2,wspace=2)

plt.subplot(8,3,1)
plt.hist(ed['ENROLL'])
plt.title("ENROLL")

plt.subplot(8,3,2)
plt.hist(ed['TOTAL_REVENUE'])
plt.title("TOTAL_REVENUE'")

plt.subplot(8,3,3)
plt.hist(ed['FEDERAL_REVENUE'])
plt.title('FEDERAL_REVENUE')

plt.subplot(8,3,4)
plt.hist(ed['STATE_REVENUE'])
plt.title("STATE_REVENUE'")

plt.subplot(8,3,5)
plt.hist(ed['LOCAL_REVENUE'])
plt.title("LOCAL_REVENUE")

plt.subplot(8,3,6)
plt.hist(ed['TOTAL_EXPENDITURE'])
plt.title("TOTAL_EXPENDITURE")

plt.subplot(8,3,7)
plt.hist(ed['INSTRUCTION_EXPENDITURE'])
plt.title("INSTRUCTION_EXPENDITURE")

plt.subplot(8,3,8)
plt.hist(ed['SUPPORT_SERVICES_EXPENDITURE'])
plt.title("SUPPORT_SERVICES_EXPENDITURE")

plt.subplot(8,3,9)
plt.hist(ed['OTHER_EXPENDITURE'])
plt.title("OTHER_EXPENDITURE")

plt.subplot(8,3,10)
plt.hist(ed['CAPITAL_OUTLAY_EXPENDITURE'])
plt.title("CAPITAL_OUTLAY_EXPENDITURE")

plt.subplot(8,3,11)
plt.hist(ed['GRADES_PK_G'])
plt.title("GRADES_PK_G")

plt.subplot(8,3,12)
plt.hist(ed['GRADES_KG_G'])
plt.title("GRADES_KG_G")

plt.subplot(8,3,13)
plt.hist(ed['GRADES_4_G'])
plt.title("GRADES_4_G")

plt.subplot(8,3,14)
plt.hist(ed['GRADES_8_G'])
plt.title("GRADES_8_G")

plt.subplot(8,3,15)
plt.hist(ed['GRADES_12_G'])
plt.title("GRADES_12_G")

plt.subplot(8,3,16)
plt.hist(ed['GRADES_1_8_G'])
plt.title("GRADES_1_8_G")

plt.subplot(8,3,17)
plt.hist(ed['GRADES_9_12_G'])
plt.title("GRADES_9_12_G")

plt.subplot(8,3,18)
plt.hist(ed['GRADES_ALL_G'])
plt.title("GRADES_ALL_G")

plt.subplot(8,3,19)
plt.hist(ed['AVG_MATH_4_SCORE'])
plt.title("AVG_MATH_4_SCORE")

plt.subplot(8,3,20)
plt.hist(ed['AVG_MATH_8_SCORE'])
plt.title('AVG_MATH_8_SCORE')

plt.subplot(8,3,21)
plt.hist(ed['AVG_READING_4_SCORE'])
plt.title('AVG_READING_4_SCORE')

plt.subplot(8,3,22)
plt.hist(ed['AVG_READING_8_SCORE'])
plt.title('AVG_READING_8_SCORE')

plt.tight_layout()
#plt.show()

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

**Boxplots for all 22 -- whis=1.5**

plt.figure(figsize=(10,20))
#plt.subplots_adjust(right=1,top=1,bottom=.2,wspace=2)

plt.subplot(8,3,1)
plt.boxplot(ed['ENROLL'],whis=1.5)
plt.title("ENROLL")

plt.subplot(8,3,2)
plt.boxplot(ed['TOTAL_REVENUE'],whis=1.5)
plt.title("TOTAL_REVENUE'")

plt.subplot(8,3,3)
plt.boxplot(ed['FEDERAL_REVENUE'],whis=1.5)
plt.title('FEDERAL_REVENUE')

plt.subplot(8,3,4)
plt.boxplot(ed['STATE_REVENUE'],whis=1.5)
plt.title("STATE_REVENUE'")

plt.subplot(8,3,5)
plt.boxplot(ed['LOCAL_REVENUE'],whis=1.5)
plt.title("LOCAL_REVENUE")

plt.subplot(8,3,6)
plt.boxplot(ed['TOTAL_EXPENDITURE'],whis=1.5)
plt.title("TOTAL_EXPENDITURE")

plt.subplot(8,3,7)
plt.boxplot(ed['INSTRUCTION_EXPENDITURE'])
plt.title("INSTRUCTION_EXPENDITURE")

plt.subplot(8,3,8)
plt.boxplot(ed['SUPPORT_SERVICES_EXPENDITURE'])
plt.title("SUPPORT_SERVICES_EXPENDITURE")

plt.subplot(8,3,9)
plt.boxplot(ed['OTHER_EXPENDITURE'])
plt.title("OTHER_EXPENDITURE")

plt.subplot(8,3,10)
plt.boxplot(ed['CAPITAL_OUTLAY_EXPENDITURE'])
plt.title("CAPITAL_OUTLAY_EXPENDITURE")

plt.subplot(8,3,11)
plt.boxplot(ed['GRADES_PK_G'])
plt.title("GRADES_PK_G")

plt.subplot(8,3,12)
plt.boxplot(ed['GRADES_KG_G'])
plt.title("GRADES_KG_G")

plt.subplot(8,3,13)
plt.boxplot(ed['GRADES_4_G'])
plt.title("GRADES_4_G")

plt.subplot(8,3,14)
plt.boxplot(ed['GRADES_8_G'])
plt.title("GRADES_8_G")

plt.subplot(8,3,15)
plt.boxplot(ed['GRADES_12_G'])
plt.title("GRADES_12_G")

plt.subplot(8,3,16)
plt.boxplot(ed['GRADES_1_8_G'])
plt.title("GRADES_1_8_G")

plt.subplot(8,3,17)
plt.boxplot(ed['GRADES_9_12_G'])
plt.title("GRADES_9_12_G")

plt.subplot(8,3,18)
plt.boxplot(ed['GRADES_ALL_G'])
plt.title("GRADES_ALL_G")

plt.subplot(8,3,19)
plt.boxplot(ed['AVG_MATH_4_SCORE'])
plt.title("AVG_MATH_4_SCORE")

plt.subplot(8,3,20)
plt.boxplot(ed['AVG_MATH_8_SCORE'])
plt.title('AVG_MATH_8_SCORE')

plt.subplot(8,3,21)
plt.boxplot(ed['AVG_READING_4_SCORE'])
plt.title('AVG_READING_4_SCORE')

plt.subplot(8,3,22)
plt.boxplot(ed['AVG_READING_8_SCORE'],whis=1.5)
plt.title('AVG_READING_8_SCORE')

plt.tight_layout()
plt.show

**Winsorize all 22 features**

smaple solution did not winzorize

In [0]:
for col in fill_list2:
  ed[col]=winsorize(ed[col], (.05,.07))

**Winsorized Boxplots**

plt.figure(figsize=(10,20))
#plt.subplots_adjust(right=1,top=1,bottom=.2,wspace=2)

plt.subplot(8,3,1)
plt.boxplot(ed['ENROLL'],whis=1.5)
plt.title("ENROLL")

plt.subplot(8,3,2)
plt.boxplot(ed['TOTAL_REVENUE'],whis=1.5)
plt.title("TOTAL_REVENUE'")

plt.subplot(8,3,3)
plt.boxplot(ed['FEDERAL_REVENUE'],whis=1.5)
plt.title('FEDERAL_REVENUE')

plt.subplot(8,3,4)
plt.boxplot(ed['STATE_REVENUE'],whis=1.5)
plt.title("STATE_REVENUE'")

plt.subplot(8,3,5)
plt.boxplot(ed['LOCAL_REVENUE'],whis=1.5)
plt.title("LOCAL_REVENUE")

plt.subplot(8,3,6)
plt.boxplot(ed['TOTAL_EXPENDITURE'],whis=1.5)
plt.title("TOTAL_EXPENDITURE")

plt.subplot(8,3,7)
plt.boxplot(ed['INSTRUCTION_EXPENDITURE'])
plt.title("INSTRUCTION_EXPENDITURE")

plt.subplot(8,3,8)
plt.boxplot(ed['SUPPORT_SERVICES_EXPENDITURE'])
plt.title("SUPPORT_SERVICES_EXPENDITURE")

plt.subplot(8,3,9)
plt.boxplot(ed['OTHER_EXPENDITURE'])
plt.title("OTHER_EXPENDITURE")

plt.subplot(8,3,10)
plt.boxplot(ed['CAPITAL_OUTLAY_EXPENDITURE'])
plt.title("CAPITAL_OUTLAY_EXPENDITURE")

plt.subplot(8,3,11)
plt.boxplot(ed['GRADES_PK_G'])
plt.title("GRADES_PK_G")

plt.subplot(8,3,12)
plt.boxplot(ed['GRADES_KG_G'])
plt.title("GRADES_KG_G")

plt.subplot(8,3,13)
plt.boxplot(ed['GRADES_4_G'])
plt.title("GRADES_4_G")

plt.subplot(8,3,14)
plt.boxplot(ed['GRADES_8_G'])
plt.title("GRADES_8_G")

plt.subplot(8,3,15)
plt.boxplot(ed['GRADES_12_G'])
plt.title("GRADES_12_G")

plt.subplot(8,3,16)
plt.boxplot(ed['GRADES_1_8_G'])
plt.title("GRADES_1_8_G")

plt.subplot(8,3,17)
plt.boxplot(ed['GRADES_9_12_G'])
plt.title("GRADES_9_12_G")

plt.subplot(8,3,18)
plt.boxplot(ed['GRADES_ALL_G'])
plt.title("GRADES_ALL_G")

plt.subplot(8,3,19)
plt.boxplot(ed['AVG_MATH_4_SCORE'])
plt.title("AVG_MATH_4_SCORE")

plt.subplot(8,3,20)
plt.boxplot(ed['AVG_MATH_8_SCORE'])
plt.title('AVG_MATH_8_SCORE')

plt.subplot(8,3,21)
plt.boxplot(ed['AVG_READING_4_SCORE'])
plt.title('AVG_READING_4_SCORE')

plt.subplot(8,3,22)
plt.boxplot(ed['AVG_READING_8_SCORE'],whis=1.5)
plt.title('AVG_READING_8_SCORE')

plt.tight_layout()
plt.show

1-**Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.**

In [0]:
#ed.describe()


In [0]:
ed['weighted']=(ed['AVG_MATH_4_SCORE']+ed['AVG_MATH_8_SCORE']+ed['AVG_READING_4_SCORE']+ed['AVG_READING_8_SCORE'])/4

In [177]:
ed.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,overall_score,weighted
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN,NaN,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895,235.639070,237.250705
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665,235.441283,234.943060
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN,NaN,NaN


**What are the correlations between this newly created score variable and the expenditure types?**

**Which 1 of the expenditure types is more correlated than the others>**

In [0]:
ed_sub=ed[['weighted','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']]

In [196]:
ed_sub.corr()

,weighted,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted,1.000000,0.102427,0.115316,0.108445,0.031866,0.019185
TOTAL_EXPENDITURE,0.102427,1.000000,0.991256,0.993101,0.945620,0.930754
INSTRUCTION_EXPENDITURE,0.115316,0.991256,1.000000,0.976215,0.908104,0.894034
SUPPORT_SERVICES_EXPENDITURE,0.108445,0.993101,0.976215,1.000000,0.953297,0.915849
OTHER_EXPENDITURE,0.031866,0.945620,0.908104,0.953297,1.000000,0.920763
CAPITAL_OUTLAY_EXPENDITURE,0.019185,0.930754,0.894034,0.915849,0.920763,1.000000


In [0]:
len(list(ed_sub))

6

**INSTRUCION_EXPENDITURE HIGHEST CORRELATION**

///////////////////////////////////////////////////////////////////////////////

In [211]:
ed_sub.head()

,weighted,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
54,237.642041,3623946.0,1578889.0,1000914.0,164083.0,680139.0
55,236.093013,1376067.0,782791.0,386526.0,68617.0,97824.0
57,244.450174,3028305.0,1537714.0,954032.0,92163.0,318959.0
58,248.119618,4079943.0,2302852.0,1190914.0,143312.0,186802.0
59,237.766088,694534.0,394680.0,207850.0,26548.0,47372.0


In [0]:
#df[df.columns[2]]
p=ed_sub[ed_sub.columns[1:6]]

In [213]:
p.head()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
54,3623946.0,1578889.0,1000914.0,164083.0,680139.0
55,1376067.0,782791.0,386526.0,68617.0,97824.0
57,3028305.0,1537714.0,954032.0,92163.0,318959.0
58,4079943.0,2302852.0,1190914.0,143312.0,186802.0
59,694534.0,394680.0,207850.0,26548.0,47372.0


X = StandardScaler().fit_transform(p)
Xt = X.T
Cx = np.cov(Xt)

In [0]:
#Cx

In [0]:
# Calculating eigenvalues and eigenvectors.
eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)

In [0]:
#eig_vec_cov 

3-**Now, apply PCA to the 5 expenditure types.**

**How much of the total variance is explained by the 1st component?**

In [0]:
#
X = StandardScaler().fit_transform(p)
sklearn_pca = PCA(n_components=1)
ed["pc1"] = sklearn_pca.fit_transform(X)

In [235]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9541168]


**95.4%**

In [0]:
#Y_sklearn

P = eig_vec_cov[:, 0]

# Transform X into Y.
Y = P.T.dot(Xt)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


4 - **What is the correlation between the overall score variable and the 1st principal component?**

In [0]:
ed_sub2=ed[['weighted','pc1','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']]

In [244]:
ed_sub2.corr()

,weighted,pc1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted,1.000000,0.072534,0.096278,0.109277,0.102267,0.030638,0.013202
pc1,0.072534,1.000000,0.995210,0.975856,0.990651,0.968760,0.952869
TOTAL_EXPENDITURE,0.096278,0.995210,1.000000,0.990829,0.993175,0.946483,0.928576
INSTRUCTION_EXPENDITURE,0.109277,0.975856,0.990829,1.000000,0.975867,0.907235,0.889763
SUPPORT_SERVICES_EXPENDITURE,0.102267,0.990651,0.993175,0.975867,1.000000,0.953789,0.913680
OTHER_EXPENDITURE,0.030638,0.968760,0.946483,0.907235,0.953789,1.000000,0.924197
CAPITAL_OUTLAY_EXPENDITURE,0.013202,0.952869,0.928576,0.889763,0.913680,0.924197,1.000000


**7.2%**

5. **If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?**

No -- most others have higher correlation